In [16]:
# !pip uninstall lightautoml -y
# !pip install -U --user LightAutoML-0.3.7.4-py3-none-any.whl >> none
# !pip install gensim

In [17]:
# !pip install -U pytorch_tabular

In [3]:
# !pip install transformers

In [4]:
# !pip uninstall torch -y
# !pip install torch

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import copy
import gc
import os
import time
import lightautoml

In [6]:
train = pd.read_csv("tr.csv")#[:10000]
test = pd.read_csv("te.csv")#[:1000]
test.sort_values("user_id", inplace=True)

print(len(train))

270000


In [7]:
# train = train[list(train.columns)[0: 100] + ["age"]]

In [8]:
# af_cols = list(pd.read_csv("af (1).csv", nrows=1).columns)[:-1]
# af_cols = [col for col in af_cols if col in train.columns]

# train.drop(af_cols, axis=1, inplace=True)
# test.drop(af_cols, axis=1, inplace=True)
# af = pd.read_csv("af (1).csv")
# train = train.merge(af, how='left', on='user_id')
# test = test.merge(af, how='left', on='user_id')

In [12]:
train['age'] = train['age'].astype('int')
train = train[~((train.url_host_fq == 1) & (train['url_host last'] == train['url_host first']))]
train = train[~train['is_male'].isna() & (train['is_male'].apply(lambda x: x != 'NA'))]
print(len(train))

261570


In [13]:
# gate = GatedAdditiveTreesBackbone(num_trees=10)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
import time


roles = {'target': 'is_male',
                 'drop': ['user_id', 'index', "age"]}
automl = TabularAutoML(
    task = Task(name='binary'),
    cpu_limit = 256,
    timeout=3600*48,
    gpu_ids='0',
    debug=True,
    general_params = {"use_algos": [["lgb"]]},
    reader_params = {'cv': 10, 'random_state': 43},
    lgb_params = {'default_params': {'device': "cpu", "boosting": "dart"}}
)
start_time = time.time()
oof_pred = automl.fit_predict(
        train,
        roles=roles,
        verbose = 1
    )
end_time = time.time()

[09:33:09] Stdout logging level is INFO.
[09:33:09] Task: binary

[09:33:09] Start automl preset with listed constraints:
[09:33:09] - time: 172800.00 seconds
[09:33:09] - CPU: 256 cores
[09:33:09] - memory: 16 GB

[09:33:09] Train data shape: (261570, 3431)

[09:35:35] Layer 1 train process start. Time left 172654.37 secs


In [20]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

print(f"spend time: {end_time - start_time}")
pred_ismale = oof_pred.data[: ,0]
pred_ismale[np.isnan(pred_ismale)] = 0.5
print(f"age ca roc auc: {roc_auc_score(train['is_male'], pred_ismale)}")
print(automl.create_model_str_desc())

spend time: 87009.34322166443
age ca roc auc: 0.8855290554795227
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) 


In [21]:
test_pred_ismale = automl.predict(test)
sub = pd.DataFrame()
sub['user_id'] = test.user_id
sub['dart_pred'] = test_pred_ismale.data[: ,0]
sub.sort_values("user_id", inplace=True)
sub.to_csv("test_dart_pred.csv", index=False)

sub.head()

,user_id,dart_pred
72084,6,0.104897
14368,7,0.949013
130234,9,0.096417
100995,10,0.020029
14369,11,0.921245


In [22]:
sub = pd.DataFrame()
sub['user_id'] = train.user_id
sub['dart_pred'] = oof_pred.data[: ,0]
sub.sort_values("user_id", inplace=True)
sub.to_csv("train_dart_pred.csv", index=False)

sub.head()

,user_id,dart_pred
135060,0,0.105708
242997,1,0.102044
215997,2,0.400263
189056,3,0.271151
0,4,0.057148
